<a href="https://colab.research.google.com/github/CarolFenixBr/Atividade-pratica-Big-Data/blob/main/Big_Data_Caroline_Vargas_da_Silva_Costa%3B_RU_4184359.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questão 1
### Qual o valor da soma de todos os campos “id” dos filmes classificados como negativos para o banco de dados “imdb-reviews-pt-br.csv”

Aluna: Caroline Vargas da Silva Costa
RU: 4184359

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar Java 8

! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# baixar, descompactar e configurar apache spark

!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xvf spark-3.5.1-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark


In [ ]:
import os

In [ ]:
# Configurando variaveis de ambientes

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME'] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
#criação de uma copia do dataframe
!cp /content/drive/MyDrive/imdb-reviews-pt-br.csv /content

In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
#criando sessão
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Carregue o arquivo CSV
df = spark.read.csv("imdb-reviews-pt-br.csv", header=True, quote="\"", escape="\"", encoding="UTF-8")

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
type(df)

In [ ]:
df.count()

In [ ]:
from pyspark.sql.functions import*

In [ ]:
df = df.withColumn(
    colName = 'id',
    col = regexp_replace('id','','').cast('int')
)

In [ ]:
#print do resultado
df.select('id').printSchema()

In [ ]:
#criação de variável com id e sentiment
df_Select = df.select('id','sentiment')

In [ ]:
#mostra resultado
df_Select.show()

In [ ]:
#Filtro por sentiment = neg
df_Neg = df_Select.where('sentiment == "neg" ')

In [ ]:
#Print resultado
print("Palavras filtradas com sentiment = neg")
df_Neg.show()

In [ ]:
total_negativo = df_Neg.groupBy('sentiment').agg(sum('id').alias("Total_Id_Negativo")).orderBy('sentiment').where('sentiment == "neg" ')
total_negativo.show()

+---------+-----------------+
|sentiment|Total_Id_Negativo|
+---------+-----------------+
|      neg|        459568555|
+---------+-----------------+



In [ ]:
soma_neg = total_negativo.select(col("Total_Id_Negativo")).first()[0]
print(f'O valor da soma de todos os campos id dos filmes classificados como negativos é: {soma_neg}')

O valor da soma de todos os campos id dos filmes classificados como negativos é: 459568555


Aluna: Caroline Vargas da Silva Costa
RU: 4184359

<h2> A resposta dos filmes classificados como negativos é 459568555 </h2>

Aluna: Caroline Vargas da Silva Costa
RU: 4184359

# Questão 2
### Diferença do número de palavras totais de português para inglês dos textos negativos

Aluna: Caroline Vargas da Silva Costa
RU: 4184359

In [ ]:
#selecionando a coluna de en e pt
df_en = df.select('id', 'text_en','sentiment')
df_pt = df.select('id', 'text_pt','sentiment')

In [ ]:
#filtrando pelo sentiment neg
df_en = df_en.where('sentiment == "neg" ')
df_pt = df_pt.where('sentiment == "neg" ')

In [ ]:
#removendo coluna
df_en = df_en.drop('sentiment')
df_pt = df_pt.drop('sentiment')

In [ ]:
df_en.show()
df_pt.show()

In [ ]:
#usar funções explode e split
palavras_en = df_en.select("id", explode(split("text_en", "[ ]+")).alias("palavra"))
palavras_pt = df_pt.select("id", explode(split("text_pt", "[ ]+")).alias("palavra"))

In [ ]:
#usar count para contar
contagem_en = palavras_en.groupBy("palavra").count().orderBy(desc("count"))
contagem_pt = palavras_pt.groupBy("palavra").count().orderBy(desc("count"))

In [ ]:
#print do resultado
print("Palavras em Ingles")
contagem_en.show()
print("Palavras em Portugues")
contagem_pt.show()

In [ ]:
#soma dos valores de palavras em pt e en
contagem_en.agg(sum('count').alias("Total_palavras_en")).show()
contagem_pt.agg(sum('count').alias("Total_palavras_pt")).show()

+-----------------+
|Total_palavras_en|
+-----------------+
|          5400276|
+-----------------+

+-----------------+
|Total_palavras_pt|
+-----------------+
|          5455273|
+-----------------+



In [ ]:
#valores para subtração
total_en = contagem_en.agg(sum('count').alias("Total_palavras_en")).first()[0]
total_pt = contagem_pt.agg(sum('count').alias("Total_palavras_pt")).first()[0]

In [ ]:
total_pt, total_en

(5455273, 5400276)

In [ ]:
subtracao = total_pt - total_en

In [ ]:
print(f'O texto em Português possui {subtracao} palavras a mais que o inglês')

O texto em Português possui 54997 palavras a mais que o inglês


Aluna: Caroline Vargas da Silva Costa
RU: 4184359

<h2> O texto em português possui 54997 palavras a mais que o inglês</h2>